In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
macro = pd.read_excel("macroeco.xlsx")

In [9]:
PE = pd.read_excel("features.xlsm", sheet_name = "PE")
PCF= pd.read_excel("features.xlsm", sheet_name = "PCF")
DIV= pd.read_excel("features.xlsm", sheet_name = "DivYield")
PB= pd.read_excel("features.xlsm", sheet_name = "PB")
DEBTEQ= pd.read_excel("features.xlsm", sheet_name = "DebtEq")
MKTVAL= pd.read_excel("features.xlsm", sheet_name = "MktVal")
#target 
Cash_Flow = pd.read_excel("features.xlsm", sheet_name = "Cash_Flow")
Prof_Margin = pd.read_excel("features.xlsm", sheet_name = "PROF_MARGIN")
Oper_Margin = pd.read_excel("features.xlsm", sheet_name = "OPER_MARGIN")

In [10]:
def process(df, type_val):
    df = df.set_index(['Unnamed: 0'])
    df = df[df.columns[::2]]
    df.reset_index(inplace = True)
    df = df.rename(columns = {'Unnamed: 0' :"index"})
    df.loc[0,:"index"] = "2008-03-31 00:00:00"
    df["index"] = pd.to_datetime(df["index"]) 
    df = df.melt(id_vars= ["index"], var_name = "Country", value_name = "Value")
    df = df.rename(columns = {'index' :"Date", 'Country': "Company", 'Value' : type_val})
    return df

PE_melt = process(PE, "PE")
PCF_melt = process(PCF, "PCF")
DIV_melt = process(DIV, "DIV")
PB_melt = process(PB, "PB")
DEBTEQ_melt = process(DEBTEQ, "DEBTEQ")
MKTVAL_melt = process(MKTVAL, "MKTVAL")
Cash_Flow_melt = process(Cash_Flow, "Cash_Flow")
Prof_Margin_melt = process(Prof_Margin, "Prof_Margin")
Oper_Margin_melt = process(Oper_Margin, "Oper_Margin")

In [11]:
data_CF = PE_melt.merge(PCF_melt, on = ["Date", "Company"], how = "outer")
data_CF = data_CF.merge(DIV_melt, on = ["Date", "Company"], how = "outer")
data_CF = data_CF.merge(PB_melt, on = ["Date", "Company"], how = "outer")
data_CF= data_CF.merge(DEBTEQ_melt, on = ["Date", "Company"], how = "outer")
data_CF = data_CF.merge(MKTVAL_melt, on = ["Date", "Company"], how = "outer")
data_CF = data_CF.merge(macro, on = ["Date"], how = "outer")
data_CF = data_CF.merge(Cash_Flow_melt, on = ["Date", "Company"], how = "outer")


In [12]:
data_PM = PE_melt.merge(PCF_melt, on = ["Date", "Company"], how = "outer")
data_PM = data_PM.merge(DIV_melt, on = ["Date", "Company"], how = "outer")
data_PM = data_PM.merge(PB_melt, on = ["Date", "Company"], how = "outer")
data_PM= data_PM.merge(DEBTEQ_melt, on = ["Date", "Company"], how = "outer")
data_PM = data_PM.merge(MKTVAL_melt, on = ["Date", "Company"], how = "outer")
data_PM = data_PM.merge(macro, on = ["Date"], how = "outer")
data_PM = data_PM.merge(Prof_Margin_melt, on = ["Date", "Company"], how = "outer")



In [13]:
data_OM = PE_melt.merge(PCF_melt, on = ["Date", "Company"], how = "outer")
data_OM = data_OM.merge(DIV_melt, on = ["Date", "Company"], how = "outer")
data_OM = data_OM.merge(PB_melt, on = ["Date", "Company"], how = "outer")
data_OM= data_OM.merge(DEBTEQ_melt, on = ["Date", "Company"], how = "outer")
data_OM = data_OM.merge(MKTVAL_melt, on = ["Date", "Company"], how = "outer")
data_OM = data_OM.merge(macro, on = ["Date"], how = "outer")
data_OM = data_OM.merge(Oper_Margin_melt, on = ["Date", "Company"], how = "outer")

In [14]:
# Move the data down by one quarter and add to data frame 
data_CF["Company"] = data_CF["Company"].astype(str)
data_CF_moved= data_CF.sort_values(by = ["Company", "Date"]).reset_index(drop = True)
data_CF_moved["CF_Target"] = data_CF_moved.groupby(["Company"])["Cash_Flow"].shift(-4)
data_CF_moved  = data_CF_moved.drop(["Cash_Flow"],axis = 1)

data_PM["Company"] = data_PM["Company"].astype(str)
data_PM_moved= data_PM.sort_values(by = ["Company", "Date"]).reset_index(drop = True)
data_PM_moved["PM_Target"] = data_PM_moved.groupby(["Company"])["Prof_Margin"].shift(-4)
data_PM_moved  = data_PM_moved.drop(["Prof_Margin"],axis = 1)

data_OM["Company"] = data_OM["Company"].astype(str)
data_OM_moved= data_OM.sort_values(by = ["Company", "Date"]).reset_index(drop = True)
data_OM_moved["OM_Target"] = data_OM_moved.groupby(["Company"])["Oper_Margin"].shift(-4)
data_OM_moved  = data_OM_moved.drop(["Oper_Margin"],axis = 1)

In [15]:
data_CF_moved = data_CF_moved.dropna(subset=["CF_Target","Date"])
data_PM_moved = data_PM_moved.dropna(subset=["PM_Target","Date"])
data_OM_moved = data_OM_moved.dropna(subset=["OM_Target","Date"])


In [16]:
data_CF_moved.to_csv("CashFlow_withmissing_1yearshift.csv")
data_PM_moved.to_csv("ProfitMargin_withmissing_1yearshift.csv")
data_OM_moved.to_csv("OperMargin_withmissing_1yearshift.csv")

In [ ]:
#missing values not filled in 